## Solving the homework

In [ ]:
from tqdm.auto import tqdm

## Anwers

Q1: Version build value is `42f05b9372a9a4a470db3b52817899b99a76ee73`.

Q2: index

Q3: 84.050095

Q4: How do I copy files from a different folder into docker container’s working directory?'

Q5: 1462

Q6: 322

## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?


## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```



In [19]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`



In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-ps2"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-ps2'})

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:04<00:00, 216.22it/s]



## Q3. Searching

Now let's search in our index. 

For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result? 

Use only `question` and `text` fields and give `question` a boost of 4

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.



In [39]:
query = "How do I execute a command in a running docker container?"

In [40]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [41]:
response['hits']['hits'][0]['_score']

84.050095

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?



In [58]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [59]:
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. Let's use these templates:


```python
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

Separate context entries by two linebreaks (`\n\n`)

Put the data from Q4 there. 

What's the length of the result? (use the `len` function)

* 962
* 1462
* 1962
* 2462



In [60]:
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [61]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [62]:
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [63]:
prompt = build_prompt(query, result_docs)

In [64]:
len(prompt)

1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```



In [65]:
import tiktoken

In [66]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [68]:
encoded_prompt = encoding.encode(prompt)

In [69]:
len(encoded_prompt)

322

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.



In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
llm_response = llm(prompt)

In [71]:
print(llm_response)

To execute a command in a running Docker container, follow these steps:

1. **Find the Container ID**:
   First, you need to identify the running container in which you want to execute the command. To do this, use the `docker ps` command to list all running containers and their respective IDs.

   ```sh
   docker ps
   ```

2. **Execute the Command**:
   Once you have the container ID, use the `docker exec` command to run your desired command within the container. For example, to start a bash session in the container, use the following command:

   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of the container.

Here is a practical example:
   
   ```sh
   docker ps
   # Output might look like this:
   # CONTAINER ID   IMAGE          COMMAND            CREATED
   # 1e5a1b663052   my_container   "some_command"     2 hours ago

   docker exec -it 1e5a1b663052 bash
   ```

This will open an interactive bash session inside the running cont

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.



In [74]:
input_price_per_k = 0.005
output_price_per_k = 0.015

average_input_length = 150
average_output_length = 250

number_of_requests = 1000

In [77]:
input_cost = input_price_per_k * average_input_length / 1000 * number_of_requests
output_cost = output_price_per_k * average_output_length / 1000 * number_of_requests

In [78]:
input_cost, output_cost, input_cost + output_cost

(0.75, 3.75, 4.5)

In [81]:
input_cost = input_price_per_k * len(encoded_prompt) / 1000 * number_of_requests
output_cost = output_price_per_k * len(encoding.encode(llm_response)) / 1000 * number_of_requests

In [82]:
input_cost, output_cost, input_cost + output_cost

(1.61, 3.675, 5.285)


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.|